In [1]:
# STILL WIP

# # Read from:
# Dune - OP Deployed by deployer address type
# Defillama/Subgraphs - TVL Flows by Program
# Notion - OP Budget by Program

# Join these datasets together on program & associate anything else to the generalized programs

In [2]:
import pandas as pd
import numpy as np
import datetime

In [3]:
tvl = pd.read_csv("csv_outputs/op_summer_latest_stats.csv")
distrib_df = pd.read_csv("csv_outputs/dune_op_distribution_type.csv")
program_df = pd.read_csv("inputs/op_incentive_program_info.csv")
app_df = pd.read_csv("csv_outputs/dune_usage_by_app.csv")

In [4]:
# Filter TVL DF
tvl = tvl[tvl["include_in_summary"] == 1]
tvl["join_key"] = tvl["top_level_name"].str.replace(
    "*", ""
)  # tvl['app_name'] + ' - ' + tvl['top_level_name'].str.replace('*','')
# display(tvl)

/var/folders/80/jqw6kmgj7j34j0x4mcs6hcl00000gn/T/ipykernel_67421/2039799251.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  tvl["join_key"] = tvl["top_level_name"].str.replace(


In [5]:
op_token_columns = [
    "op_claimed",
    "op_deployed",
    "op_from_other_projects",
    "op_to_other_projects",
    "op_to_project",
]

In [6]:
# Set up Distributions for Mapping
distrib_df["program_map"] = np.where(
    distrib_df["counterparty_type"].isin(tvl["top_level_name"]),
    distrib_df["counterparty_type"],
    "",
)
group_cols = [
    "project_name",
    "counterparty_label",
    "counterparty_type",
    "program_map",
] + op_token_columns

sum_distrib_df = distrib_df[group_cols].groupby(["project_name", "program_map"]).sum()
sum_distrib_df.reset_index(inplace=True)
# Joins should maybe just be the program map OR from name, since Velo operated bribes for a while
sum_distrib_df["join_key"] = np.where(
    sum_distrib_df["program_map"] == "",
    sum_distrib_df["project_name"],
    sum_distrib_df["program_map"],
)
# sum_distrib_df['from_name'] + ' - ' \
#     + np.where(sum_distrib_df['program_map'] == '',sum_distrib_df['from_name'],sum_distrib_df['program_map'])

# display(sum_distrib_df[sum_distrib_df['join_key'].str.contains('elodr')])

In [7]:
# lowercase joinkeys
tvl["join_key"] = tvl["join_key"].str.lower()
sum_distrib_df["join_key"] = sum_distrib_df["join_key"].str.lower()
#
df = sum_distrib_df.merge(tvl, on="join_key", how="outer")
# display(df[df['join_key'].str.contains('velodr')])

In [8]:
# Overrides as needed
def replace_program_names(df, overrides):
    for program, program_override in overrides.items():
        df.loc[df["join_key"] == program, "from_name"] = program_override
    return df


# Overrides if needed
overrides = {
    "old name": "new name",
}

# Replace program names with overrides
df = replace_program_names(df, overrides)

In [9]:
# Create the aggregate app name field
df["agg_app_name"] = df["app_name"].combine_first(df["project_name"])
df = df.fillna(0)  # Fill NA with 0
# display(df)

In [10]:
# Now union back again

data_cols = [
    "agg_app_name",
    "top_level_name",
    "program_name",
    "num_op_override",
    "period",
    "op_source",
    "start_date",
    "end_date",
    "cumul_net_dollar_flow_at_program_end",
    "cumul_net_dollar_flow",
    "cumul_last_price_net_dollar_flow_at_program_end",
    "cumul_last_price_net_dollar_flow",
]
select_cols = data_cols + op_token_columns

group_cols = select_cols[:8]  # group by 1 to 8
print(group_cols)

# display(df[select_cols])

sum_distrib_df = df[select_cols].groupby(group_cols).sum()
sum_distrib_df.reset_index(inplace=True)

['agg_app_name', 'top_level_name', 'program_name', 'num_op_override', 'period', 'op_source', 'start_date', 'end_date']


In [11]:
# Get the rank by start_date of each program

sum_distrib_df = sum_distrib_df.reset_index().rename(columns={"index": "row_num"})

# replace 0s with '9999-12-31'
sum_distrib_df["start_date"] = np.where(
    sum_distrib_df["start_date"] == 0, "9999-12-31", sum_distrib_df["start_date"]
)
# create a new column 'program_rank' based on the 'start_date' column
sum_distrib_df = sum_distrib_df.sort_values(["agg_app_name", "start_date", "row_num"])
sum_distrib_df["program_rank"] = sum_distrib_df.groupby("agg_app_name").cumcount() + 1

sum_distrib_df = sum_distrib_df.sort_values(
    by=["agg_app_name", "program_rank"], ascending=[True, True]
)
# subtract all overridden values fromthe amount I have deployed

# create a new column 'cumulative_num_op_override' that contains the cumulative sum of 'num_op_override' for each agg_app_name group
sum_distrib_df["cumulative_num_op_override"] = sum_distrib_df.groupby("agg_app_name")[
    "num_op_override"
].cumsum()


# create a new column 'op_deployed_net_override' that subtracts 'cumulative_num_op_override' from 'op_deployed'
sum_distrib_df["op_deployed_net_override"] = sum_distrib_df[
    "op_deployed"
] - sum_distrib_df.groupby("agg_app_name")["cumulative_num_op_override"].shift(
    1
).fillna(
    0
)
# drop the 'cumulative_num_op_override' column
# sum_distrib_df.drop('cumulative_num_op_override', axis=1, inplace=True)

# replace '9999-12-31' with 0s
sum_distrib_df["start_date"] = np.where(
    sum_distrib_df["start_date"] == "9999-12-31", 0, sum_distrib_df["start_date"]
)

# Drop Row Num
sum_distrib_df.drop("row_num", axis=1, inplace=True)
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

In [12]:
# Now do the algorithmic overrides - where we want to redistirbute deployed OP across specific programs (i.e. Uniswap LM w/ Partners)
# # replace 0s in 'num_op_override' with the corresponding value in 'op_deployed_net_override'
sum_distrib_df["og_op_deployed"] = sum_distrib_df["op_deployed"]
# Override # OP Deployed
sum_distrib_df["op_deployed"] = np.where(
    (sum_distrib_df["num_op_override"] == 0),
    sum_distrib_df["op_deployed_net_override"],
    sum_distrib_df["num_op_override"],
)

# Hardcode for Aave - Liquidity Mining since claims came straight from the FND wallet. This should be a one-time edge case
sum_distrib_df["op_deployed"] = np.where(
    sum_distrib_df["top_level_name"] == "Aave - Liquidity Mining",
    5_000_000,
    sum_distrib_df["op_deployed"],
)

In [13]:
# #Select all except the last 4 rows
sum_distrib_df = sum_distrib_df.iloc[:, :-4]

In [14]:
# latest tvl metrics
sum_distrib_df["cumul_last_price_net_dollar_flow_at_program_end_ended"] = np.where(
    sum_distrib_df["period"] == "Post-Program",
    sum_distrib_df["cumul_last_price_net_dollar_flow_at_program_end"],
    np.nan,
)

sum_distrib_df["cumul_last_price_net_dollar_flow_ended"] = np.where(
    sum_distrib_df["period"] == "Post-Program",
    sum_distrib_df["cumul_last_price_net_dollar_flow"],
    np.nan,
)
sum_distrib_df["net_flows_retention"] = np.where(
    sum_distrib_df["period"] == "Post-Program",
    sum_distrib_df["cumul_last_price_net_dollar_flow_ended"]
    / sum_distrib_df["cumul_last_price_net_dollar_flow_at_program_end_ended"],
    np.nan,
)
# If < 0 then make retention 0
sum_distrib_df["net_flows_retention"] = np.where(
    sum_distrib_df["cumul_last_price_net_dollar_flow"] < 0,
    0,
    sum_distrib_df["net_flows_retention"],
)

# Live Programs

sum_distrib_df["cumul_last_price_net_dollar_flow_at_program_end_live"] = np.where(
    sum_distrib_df["period"] == "During Program",
    sum_distrib_df["cumul_last_price_net_dollar_flow_at_program_end"],
    np.nan,
)

sum_distrib_df["cumul_last_price_net_dollar_flow_live"] = np.where(
    sum_distrib_df["period"] == "During Program",
    sum_distrib_df["cumul_last_price_net_dollar_flow"],
    np.nan,
)

In [15]:
# Get App Name Mappings from Notion
name_mappings = program_df[["App Name", "App Name Map Override"]].drop_duplicates()
name_mappings = name_mappings[~name_mappings["App Name Map Override"].isna()]
name_mappings = name_mappings.rename(columns={"App Name": "agg_app_name"})
sum_distrib_df = sum_distrib_df.merge(name_mappings, on="agg_app_name", how="left")
sum_distrib_df["App Name Map"] = (
    sum_distrib_df["App Name Map Override"].combine_first(
        sum_distrib_df["agg_app_name"]
    )
).str.lower()

sum_distrib_df["last_updated"] = pd.to_datetime(datetime.datetime.now())

# Only export apps with a TVL
sum_distrib_df_export = sum_distrib_df[sum_distrib_df["top_level_name"] != 0]
sum_distrib_df_export.to_csv("csv_outputs/incentives_stats_summary.csv")
display(sum_distrib_df_export)
# display(sum_distrib_df[sum_distrib_df['agg_app_name'].str.contains('rrakis')])

,agg_app_name,top_level_name,program_name,num_op_override,period,op_source,start_date,end_date,cumul_net_dollar_flow_at_program_end,cumul_net_dollar_flow,...,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live,App Name Map Override,App Name Map,last_updated
1,Aave,Aave - Liquidity Mining,Aave V3 - Aave - Liquidity Mining,0.0,Post-Program,Partner Fund,2022-08-04,2022-11-04,3.450056e+08,7.849483e+07,...,604.691758,0.000000e+00,3.419694e+08,7.732963e+07,0.226130,NaN,NaN,NaN,aave,2023-04-04 17:39:16.279497
3,Aelin,Aelin,Aelin,0.0,Post-Program,Governance - Phase 0,2022-09-12,2022-09-14,-2.827652e+03,-2.778652e+03,...,19827.000000,9.000000e+05,-2.826693e+03,-2.777707e+03,0.000000,NaN,NaN,NaN,aelin,2023-04-04 17:39:16.279497
5,Alchemix,alETH/WETH: Velodrome,Alchemix - alETH/WETH: Velodrome,0.0,During Program,Governance - Season 1,2023-01-08,0,8.609117e+06,8.609117e+06,...,0.000000,0.000000e+00,NaN,NaN,NaN,8.566602e+06,8.566602e+06,NaN,alchemix,2023-04-04 17:39:16.279497
7,Angle,OP/agEUR: Velodrome,Angle - OP/agEUR: Velodrome,0.0,During Program,Governance - Season 1,2023-02-20,0,3.695839e+05,3.695839e+05,...,0.000000,0.000000e+00,NaN,NaN,NaN,3.626476e+05,3.626476e+05,NaN,angle,2023-04-04 17:39:16.279497
8,Angle,USDC/agEUR: Uniswap,Angle - USDC/agEUR: Uniswap,0.0,During Program,Governance - Season 1,2023-02-20,0,-1.109531e+04,-1.109531e+04,...,0.000000,0.000000e+00,NaN,NaN,NaN,8.646041e+03,8.646041e+03,NaN,angle,2023-04-04 17:39:16.279497
9,Angle,agEUR/MAI: Velodrome,Angle - agEUR/MAI: Velodrome,0.0,During Program,Governance - Season 1,2023-02-20,0,-7.434744e+05,-7.434744e+05,...,0.000000,0.000000e+00,NaN,NaN,0.000000,-8.769065e+05,-8.769065e+05,NaN,angle,2023-04-04 17:39:16.279497
11,Arrakis Finance,Uniswap LM - Phase 1*,Arrakis Finance - Uniswap LM - Phase 1*,16667.0,Post-Program,Governance - Phase 0,2022-10-26,2022-11-21,-5.779462e+05,-3.095715e+05,...,0.000000,0.000000e+00,-5.779462e+05,-3.095715e+05,0.000000,NaN,NaN,NaN,arrakis finance,2023-04-04 17:39:16.279497
12,Arrakis Finance,Uniswap LM - Phase 2*,Arrakis Finance - Uniswap LM - Phase 2*,25000.0,Post-Program,Governance - Phase 0,2023-01-16,2023-02-08,5.039975e+06,-1.427338e+06,...,0.000000,0.000000e+00,5.039975e+06,-1.427338e+06,0.000000,NaN,NaN,NaN,arrakis finance,2023-04-04 17:39:16.279497
13,Arrakis Finance,OP Uniswap LM - Phase 1*,Arrakis Finance - OP Uniswap LM - Phase 1*,67500.0,During Program,Partner Fund,2023-03-22,2023-04-12,1.417283e+07,1.417283e+07,...,0.000000,0.000000e+00,NaN,NaN,NaN,1.417283e+07,1.417283e+07,NaN,arrakis finance,2023-04-04 17:39:16.279497
16,Beefy Finance,BIFI/OP: Velodrome,Beefy - BIFI/OP: Velodrome,0.0,During Program,Governance - Season 1,2022-09-13,0,5.992491e+05,5.992491e+05,...,0.000000,0.000000e+00,NaN,NaN,NaN,7.674874e+05,7.674874e+05,NaN,beefy finance,2023-04-04 17:39:16.279497


In [16]:
# sum_distrib_df_grouped = sum_distrib_df

In [17]:
# assuming your dataframe is named `df`
program_df["App Name Map"] = (
    program_df["App Name Map Override"].combine_first(program_df["App Name"])
).str.lower()
program_df_grouped = (
    program_df.groupby("App Name Map")
    .agg({"# OP Allocated": "sum", "Source": lambda x: list(set(x))})
    .reset_index()
)

# reset_index() is used to convert the grouped result back to a dataframe
# the lambda function for 'Source' column aggregates text entries as a list/array
# display(program_df_grouped)

# Get Deployments Grouped
sum_distrib_df_grouped = sum_distrib_df.groupby("App Name Map").agg(sum).reset_index()
sum_distrib_df_grouped = sum_distrib_df_grouped.reset_index()
sum_distrib_df_grouped["net_flows_retention"] = np.where(
    sum_distrib_df_grouped["cumul_last_price_net_dollar_flow_ended"] < 0,
    0,
    sum_distrib_df_grouped["cumul_last_price_net_dollar_flow_ended"]
    / sum_distrib_df_grouped["cumul_last_price_net_dollar_flow_at_program_end_ended"],
)
sum_distrib_df_grouped.drop(
    [
        "num_op_override",
        "cumul_net_dollar_flow_at_program_end",
        "cumul_net_dollar_flow",
    ],
    axis=1,
    inplace=True,
)

# purpose is to make sure we don't double count tokens out
# From the app presepctive, we look at deployed
# From the overall / by season prespective, we look at net
sum_distrib_df_grouped["op_net_deployed"] = (
    sum_distrib_df_grouped["op_deployed"]
    - sum_distrib_df_grouped["op_from_other_projects"]
)

display(sum_distrib_df_grouped)
sum_distrib_df_grouped.to_csv("csv_outputs/incentives_summary_by_app.csv")
print(sum_distrib_df_grouped["op_net_deployed"].sum())

print(sum_distrib_df_grouped.columns)

,index,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live,op_net_deployed
0,0,1inch,0.000000e+00,0.000000e+00,2.696986e+04,3.000000e+05,0.00,0.000000,300000.000000,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,3.000000e+05
1,1,aave,3.419694e+08,7.732963e+07,4.822990e+06,5.000000e+06,0.00,604.691758,300000.000000,3.419694e+08,7.732963e+07,0.226130,0.000000e+00,0.000000e+00,5.000000e+06
2,2,aelin,-2.826693e+03,-2.777707e+03,8.801740e+05,9.000010e+05,0.00,19827.000000,900000.000000,-2.826693e+03,-2.777707e+03,0.000000,0.000000e+00,0.000000e+00,9.000010e+05
3,3,airdrop #1 distributor,0.000000e+00,0.000000e+00,1.639225e+08,1.639294e+08,0.00,0.000000,0.000000,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,1.639294e+08
4,4,alchemix,8.566602e+06,8.566602e+06,3.109853e+04,3.312140e+04,0.00,0.000000,250000.000000,0.000000e+00,0.000000e+00,NaN,8.566602e+06,8.566602e+06,3.312140e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,74,wepiggy,8.331370e+05,1.667840e+05,2.996520e+05,3.000013e+05,0.00,0.000000,300008.466958,8.331370e+05,1.667840e+05,0.200188,0.000000e+00,0.000000e+00,3.000013e+05
75,75,xtoken,4.244216e+06,2.409923e+05,4.886695e+06,4.255042e+05,83333.32,0.000000,600007.965054,5.165498e+06,1.162274e+06,0.225007,-9.212815e+05,-9.212815e+05,3.421709e+05
76,76,yearn,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000,1000000.000000,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00
77,77,zeroex (0x),0.000000e+00,0.000000e+00,4.520000e+02,4.520000e+02,0.00,0.000000,300000.000000,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,4.520000e+02


234670024.30208635
Index(['index', 'App Name Map',
       'cumul_last_price_net_dollar_flow_at_program_end',
       'cumul_last_price_net_dollar_flow', 'op_claimed', 'op_deployed',
       'op_from_other_projects', 'op_to_other_projects', 'op_to_project',
       'cumul_last_price_net_dollar_flow_at_program_end_ended',
       'cumul_last_price_net_dollar_flow_ended', 'net_flows_retention',
       'cumul_last_price_net_dollar_flow_at_program_end_live',
       'cumul_last_price_net_dollar_flow_live', 'op_net_deployed'],
      dtype='object')


In [18]:
joined_df = sum_distrib_df_grouped.merge(
    program_df_grouped, on="App Name Map", how="outer"
)
display(joined_df)

,index,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,cumul_last_price_net_dollar_flow_ended,net_flows_retention,cumul_last_price_net_dollar_flow_at_program_end_live,cumul_last_price_net_dollar_flow_live,op_net_deployed,# OP Allocated,Source
0,0.0,1inch,0.000000e+00,0.000000e+00,2.696986e+04,3.000000e+05,0.0,0.000000,300000.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,3.000000e+05,300000.0,[Governance - Phase 0]
1,1.0,aave,3.419694e+08,7.732963e+07,4.822990e+06,5.000000e+06,0.0,604.691758,300000.0,3.419694e+08,7.732963e+07,0.22613,0.000000e+00,0.000000e+00,5.000000e+06,5300000.0,"[Governance - Phase 0, Partner Fund]"
2,2.0,aelin,-2.826693e+03,-2.777707e+03,8.801740e+05,9.000010e+05,0.0,19827.000000,900000.0,-2.826693e+03,-2.777707e+03,0.00000,0.000000e+00,0.000000e+00,9.000010e+05,900000.0,[Governance - Phase 0]
3,3.0,airdrop #1 distributor,0.000000e+00,0.000000e+00,1.639225e+08,1.639294e+08,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,1.639294e+08,NaN,NaN
4,4.0,alchemix,8.566602e+06,8.566602e+06,3.109853e+04,3.312140e+04,0.0,0.000000,250000.0,0.000000e+00,0.000000e+00,NaN,8.566602e+06,8.566602e+06,3.312140e+04,250000.0,[Governance - Season 2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,NaN,opweave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8633.0,[Governance - Season 3]
100,NaN,overnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400000.0,[Governance - Season 2]
101,NaN,parcel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150000.0,[Governance - Season 3]
102,NaN,superfluid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150000.0,[Governance - Season 1]


In [19]:
app_cols = [
    "app_name",
    "txs_per_day_prev",
    "num_addr_per_day_prev",
    "gas_fee_eth_per_day_prev",
    "txs_per_day",
    "num_addr_per_day",
    "gas_fee_eth_per_day",
    "txs_per_day_after",
    "num_addr_per_day_after",
    "gas_fee_eth_per_day_after",
]
app_df_sm = app_df[app_cols]
app_df_sm["app_name"] = app_df_sm["app_name"].str.lower()
app_df_sm = app_df_sm.rename(columns={"app_name": "App Name Map"})

/var/folders/80/jqw6kmgj7j34j0x4mcs6hcl00000gn/T/ipykernel_67421/1737797060.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app_df_sm["app_name"] = app_df_sm["app_name"].str.lower()


In [20]:
joined_df = joined_df.merge(app_df_sm, on="App Name Map", how="outer")

display(joined_df)

joined_df.to_csv("csv_outputs/total_stats_summary_by_app.csv")

,index,App Name Map,cumul_last_price_net_dollar_flow_at_program_end,cumul_last_price_net_dollar_flow,op_claimed,op_deployed,op_from_other_projects,op_to_other_projects,op_to_project,cumul_last_price_net_dollar_flow_at_program_end_ended,...,Source,txs_per_day_prev,num_addr_per_day_prev,gas_fee_eth_per_day_prev,txs_per_day,num_addr_per_day,gas_fee_eth_per_day,txs_per_day_after,num_addr_per_day_after,gas_fee_eth_per_day_after
0,0.0,1inch,0.000000e+00,0.000000e+00,2.696986e+04,3.000000e+05,0.0,0.000000,300000.0,0.000000e+00,...,[Governance - Phase 0],1137.500000,408.700000,1.577298,3156.000000,759.000000,4.221651,1522.689655,756.965517,1.537912
1,1.0,aave,3.419694e+08,7.732963e+07,4.822990e+06,5.000000e+06,0.0,604.691758,300000.0,3.419694e+08,...,"[Governance - Phase 0, Partner Fund]",247.733333,125.966667,0.412240,1648.462366,943.978495,4.720679,2439.862069,1466.000000,8.215421
2,2.0,aelin,-2.826693e+03,-2.777707e+03,8.801740e+05,9.000010e+05,0.0,19827.000000,900000.0,-2.826693e+03,...,[Governance - Phase 0],9.700000,6.100000,0.072883,15.666667,9.000000,0.104011,4.965517,3.620690,0.033766
3,3.0,airdrop #1 distributor,0.000000e+00,0.000000e+00,1.639225e+08,1.639294e+08,0.0,0.000000,0.0,0.000000e+00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,alchemix,8.566602e+06,8.566602e+06,3.109853e+04,3.312140e+04,0.0,0.000000,250000.0,0.000000e+00,...,[Governance - Season 2],6.833333,3.500000,0.054973,14.813953,8.174419,0.198721,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,NaN,opweave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[Governance - Season 3],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,overnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[Governance - Season 2],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,parcel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[Governance - Season 3],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,superfluid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,[Governance - Season 1],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
